In [1]:
#Then calculate the K.

import Spectrum_mol as Spec
import numpy as np
import matplotlib.pyplot as plt
import Spectrum_mol as Spec
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
def calculate_spectral_overlap(x1,y1, x2,y2, diff):

    """
    Energy difference is neglected?
    
    """
    #initializing intergral.
    #print("initializing intergral")
    intergral = 0
    
    for x_index, x in enumerate(x1):
        if x >= 0 and x < 999:
            #start calculating intergral.
            intergrand = (x1[x_index+1] - x) * min(y1[x_index], y2[x_index])
            intergral += intergrand
    return intergral

def normalize_spec(xaxis, yaxis):
    """
    This function takes in spec and return the normalized spec
    numerically calculate the AUC and then normalize
    WARNING:
    FIX THE UNIT BEFORE NORMALIZATION!!!
    
    input:
    x-axis (in what unit??)
    y-axis (absorption, dimensionless)
    
    output:
    x-axis (same as input unit)
    y-axis (absorption)
    
    """

    #Calculate the total AUC
    AUC = 0
    for index in range(1,1000):
        AUC += yaxis[index] * (xaxis[index]-xaxis[index-1])
    
    #print("AUC is %s" %AUC)
    
    #Calculate the normalization factor, use this to multiply the Absorption to get normalized spec
    norm_factor = 1/AUC
    
    #Reassign the value in yaxis
    new_yaxis = []
    for index in range(0, 1000):
        new_yaxis.append(yaxis[index]*norm_factor)
    #print(np.array(new_yaxis).shape)
    #print(xaxis.shape)
    
    return (xaxis, np.array(new_yaxis))
    
def generate_AtFt_renger():
    Ft_response = np.load("./RENGER_At(0, 1.0, 0.001).npy")
    At_response = np.load("./RENGER_At(0, 1.0, 0.001).npy")
    print(At_response[0])
    normed_At_responsex, normed_At_responsey = normalize_spec(np.arange(0,1000), At_response.real)
    normed_Ft_responsex, normed_Ft_responsey = normalize_spec(np.arange(0,1000), Ft_response.real)
    
    #plt.plot(fixed_Aw_responsex, fixed_Aw_responsey)
    #plt.plot(fixed_Fw_responsex, fixed_Fw_responsey)
    #print(fixed_Fw_responsex)
    return (normed_At_responsex, normed_At_responsey, normed_Ft_responsex, normed_Ft_responsey)
    
def generate_AtFt_car2():
    Ft_response = np.load("./CAR2_Ft(0, 1.0, 0.001).npy")
    At_response = np.load("./CAR2_At(0, 1.0, 0.001).npy")
    normed_At_responsex, normed_At_responsey = normalize_spec(np.arange(0,1000), At_response.real)
    normed_Ft_responsex, normed_Ft_responsey = normalize_spec(np.arange(0,1000), Ft_response.real)
    
    #plt.plot(fixed_Aw_responsex, fixed_Aw_responsey)
    #plt.plot(fixed_Fw_responsex, fixed_Fw_responsey)
    #print(fixed_Fw_responsex)
    return (normed_At_responsex, normed_At_responsey, normed_Ft_responsex, normed_Ft_responsey)
    

In [3]:
from scipy.constants import pi 
from scipy.constants import hbar #hbar in joules*sec.
from scipy.constants import speed_of_light

#Input:ps (time) , Output: cm-1 (energy)
def ps_to_cm(time_ps):
    return(1.0/(200.0*pi*speed_of_light*(time_ps*1E-12)))


#Input: cm-1 (energy), Output: ps (time)
def cm_to_ps(energy_cm):
    return((200.00*pi*speed_of_light*energy_cm*1E-12))


def build_k_matrix():
    """
    In this function we build the transfer matrix K, with N*N elements.
    As of the terminal emitter:
    
            LUT620 CLA610 CLA611 CLA612 CHL_POOL
    LUT620 
    CLA610
    CLA611
    CLA612
    CHL_POOL
    
    diagonal elements are: K_mn = -∑_(n≠m){k_mn} -k_m^d
    off-diagonal elements: K_mn = k_nm
    so K is a 4*4 matrix, we will assembly this by hand.
    
    the chl 1/Kd can be 4ns.
    the lifetime of the S2 state is as short as 200 fs, while that of the S1 state is as long as 10 ps 
    
    
    NOTE: FOR KINETICS WE CONVERT EVERTHING TO ps^-1
    
    """

    K = np.zeros((5,5)) #initialize K matrix
    print("K matrix initialized")
    print(K)
    
    kd_cla = 1/(4.0E3) #in ps-1
    kd_lut = 1/10 #in ps-1
    kd_pool = 5*kd_cla
    
    cla610_energy = 15100 #cm^-1
    cla611_energy = 15100 #cm^-1
    cla612_energy = 15100 #cm^-1
    lut620_energy = 14900 #cm^-1
    
    
    #k_lut620cla610 = 2*pi/hbar * coupling^2 * spec_overlap
    #Here we introduce the couplings for each pair.
    #Note the "Coupling_from_tresp/coupling_calc.py" calculates the TRESP couplings in cm-1. Convert to Joules: / 5.034E22 
    #Here we convert all to Joules:
    #hbar is in J*sec
    #      constant
    #K = ------------ * constant^2 * J  ; hence its unit is in sec-1
    #       J*sec

    data = np.load('../../Coupling_from_tresp/Hamiltonian_result7.npy') #Couplings in cm^-1
    J620_610 = np.average(data[:,0,1])
    J620_611 = np.average(data[:,0,2])
    J620_612 = np.average(data[:,0,3])
    J610_611 = np.average(data[:,1,2])
    J610_612 = np.average(data[:,1,3])
    J611_612 = np.average(data[:,2,3])

    
    #Here we calculate the spec overlap for each pair
    Aw620x, Aw620y, Fw620x, Fw620y = generate_AtFt_car2()
    Aw610x, Aw610y, Fw610x, Fw610y = generate_AtFt_renger()
    Aw611x, Aw611y, Fw611x, Fw611y = generate_AtFt_renger()
    Aw612x, Aw612y, Fw612x, Fw612y = generate_AtFt_renger()

    
    #1 cm−1=1.986*10−23 J
    O620_610 = calculate_spectral_overlap(Fw620x, Fw620y, Aw610x, Aw610y, cla610_energy-lut620_energy) #energy from 620 to 610. use the Fw620, Aw610.
    O620_611 = calculate_spectral_overlap(Fw620x, Fw620y, Aw611x, Aw611y, cla611_energy-lut620_energy) #energy from 620 to 611. use the Fw620, Aw611.
    O620_612 = calculate_spectral_overlap(Fw620x, Fw620y, Aw612x, Aw612y, cla612_energy-lut620_energy) #energy from 620 to 612. use the Fw620, Aw612.
    O610_611 = calculate_spectral_overlap(Fw610x, Fw610y, Aw611x, Aw611y, cla611_energy-cla610_energy) #energy from 610 to 611. use the Fw610, Aw611.
    O610_612 = calculate_spectral_overlap(Fw610x, Fw610y, Aw612x, Aw612y, cla612_energy-cla610_energy) #energy from 610 to 612. use the Fw610, Aw612.
    O611_612 = calculate_spectral_overlap(Fw611x, Fw611y, Aw612x, Aw612y, cla612_energy-cla611_energy) #energy from 611 to 612. use the Fw611, Aw612.

    
    """
    #here convert cm-1 to joules.
    O620_610 *= 1.986E-23
    O620_611 *= 1.986E-23
    O620_612 *= 1.986E-23
    O610_611 *= 1.986E-23
    O610_612 *= 1.986E-23
    O611_612 *= 1.986E-23
    """
    
    
    #here we assembly the matrix:
    #Here K elements are in cm, convert them into ps: use the cm_to_ps function
    #off-diagnal terms
    K[0,1] = 2 * J620_610**2 * O620_610
    K[0,2] = 2 * J620_611**2 * O620_611
    K[0,3] = 2 * J620_612**2 * O620_612
    K[1,2] = 2 * J610_611**2 * O610_611
    K[1,3] = 2 * J610_612**2 * O610_612
    K[2,3] = 2 * J611_612**2 * O611_612
    
    K[0,1] = 1/cm_to_ps(K[0,1])
    K[0,2] = 1/cm_to_ps(K[0,2])
    K[0,3] = 1/cm_to_ps(K[0,3])
    K[1,2] = 1/cm_to_ps(K[1,2])
    K[1,3] = 1/cm_to_ps(K[1,3])
    K[2,3] = 1/cm_to_ps(K[2,3])    
    #print(K)
    
    K += K.T
    
    K[0,4] = 0 #lutein to pool
    K[1,4] = 1/5 #cla610 to pool
    K[2,4] = 1/5 #cla611 to pool
    K[3,4] = 1/5 #cla612 to pool
    
    K[4,0] = 0 #pool to lutein
    K[4,1] = 1/1 #pool to cla610
    K[4,2] = 1/1 #pool to cla611
    K[4,3] = 1/1 #pool to cla612
    
    print(K)

    
    #print(K)
    
    #diagnal terms
    K[0,0] = -(K[0,1] + K[0,2] + K[0,3] + K[0,4]) - kd_lut
    K[1,1] = -(K[1,0] + K[1,2] + K[1,3] + K[1,4]) - kd_cla
    K[2,2] = -(K[2,0] + K[2,1] + K[2,3] + K[2,4]) - kd_cla
    K[3,3] = -(K[3,0] + K[3,1] + K[3,2] + K[3,4]) - kd_cla
    K[4,4] = -(K[4,0] + K[4,1] + K[4,2] + K[4,3]) - kd_pool

    return K

In [4]:
build_k_matrix()

K matrix initialized
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
(1+0j)
(1+0j)
(1+0j)
[[ 0.00000000e+00 -3.15487183e-02 -6.84780735e-02 -1.25789244e-02
   0.00000000e+00]
 [-3.15487183e-02  0.00000000e+00  8.91828998e-03  6.62652125e-03
   2.00000000e-01]
 [-6.84780735e-02  8.91828998e-03  0.00000000e+00  2.73452893e-04
   2.00000000e-01]
 [-1.25789244e-02  6.62652125e-03  2.73452893e-04  0.00000000e+00
   2.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00]]


array([[ 1.26057162e-02, -3.15487183e-02, -6.84780735e-02,
        -1.25789244e-02,  0.00000000e+00],
       [-3.15487183e-02, -1.84246093e-01,  8.91828998e-03,
         6.62652125e-03,  2.00000000e-01],
       [-6.84780735e-02,  8.91828998e-03, -1.40963669e-01,
         2.73452893e-04,  2.00000000e-01],
       [-1.25789244e-02,  6.62652125e-03,  2.73452893e-04,
        -1.94571050e-01,  2.00000000e-01],
       [ 0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00, -3.00125000e+00]])